In [19]:
import numpy as np
import networkx as nx
from node2vec import Node2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm

from ruwordnet.ruwordnet_reader import RuWordnet

In [20]:
def build_graph(ruwordnet, pos="N", directed=False):
    G = nx.DiGraph() if directed else nx.Graph()

    synsets = ruwordnet.get_all_synsets(pos)  # likely list of tuples (id, name)
    for row in synsets:
        synset_id = row[0] if isinstance(row, tuple) else row  # support both formats
        G.add_node(synset_id)

        hypers = ruwordnet.get_hypernyms_by_id(synset_id)  # list of synset IDs
        for h_id in hypers:
            G.add_edge(synset_id, h_id)

    return G

ruwordnet = RuWordnet(db_path="../data/ruwordnet.db", ruwordnet_path=None)
G = build_graph(ruwordnet, pos="N", directed=False) 
print("nodes:", G.number_of_nodes(), "edges:", G.number_of_edges())

nodes: 29296 edges: 39110


In [21]:
node2vec = Node2Vec(G, dimensions=300, walk_length=20, num_walks=3, workers=1)
node2vec_model = node2vec.fit(window=10, min_count=1, epochs=7)

Generating walks (CPU: 1): 100%|██████████| 3/3 [00:09<00:00,  3.04s/it]


In [22]:
print('Saving just in case...')
node2vec_model.wv.save_word2vec_format("models/vectors/node2vec/node2vec_ru_nouns_new.txt")
node2vec_model.save("models/node2vec_en_nouns_new")

Saving just in case...


In [23]:
node2vec_model = KeyedVectors.load_word2vec_format("models/vectors/node2vec/node2vec_ru_nouns_new.txt", binary=False)
word2vec_model = KeyedVectors.load_word2vec_format("models/vectors/fasttext/ruwordnet_nouns_fasttext.txt", binary=False)

In [8]:
is_sense = any(
    synset_id == ruwordnet.get_id_by_name('ПОСЕТИТЕЛЬ')
    for sense_id, synset_id, text in tqdm(ruwordnet.get_all_senses())
)
print(is_sense)

 14%|█▍        | 18352/130417 [03:23<20:42, 90.20it/s]

True


In [9]:
is_synset = any(
    synset_id == ruwordnet.get_id_by_name('ПОСЕТИТЕЛЬ')
    for synset_id, text in tqdm(ruwordnet.get_all_synsets('N'))
)
print(is_synset)

  0%|          | 12/29296 [00:00<03:15, 149.43it/s]

True


In [8]:
vector = word2vec_model[f'{ruwordnet.get_id_by_name("ПОСЕТИТЕЛЬ")}']
print(vector[:5])

[ 0.04510555 -0.02949081 -0.01271916  0.00454717  0.03544347]


In [24]:
def print_synset_summary(wn, query_name):
    print(f"Synset: {query_name}",)
    query_id = wn.get_id_by_name(query_name)
    hypernym_ids = wn.get_hypernyms_by_id(query_id)
    for hypernym_id in hypernym_ids:
        hypernym_name = wn.get_name_by_id(hypernym_id)
        print(f"Query's hypernyms: {hypernym_name}")
    print("-"*10)

print_synset_summary(wn=ruwordnet, query_name='ПОСЕТИТЕЛЬ')
print("Most similar words (node2vec):")
for synset_id, similarity in node2vec_model.most_similar(ruwordnet.get_id_by_name('ПОСЕТИТЕЛЬ'), topn=10):
     synset_name = ruwordnet.get_name_by_id(synset_id)
     print(synset_name, similarity, ruwordnet.get_name_by_id(ruwordnet.get_hypernyms_by_id(synset_id)[0]))
print("-"*10)
for synset_id, similarity in word2vec_model.most_similar(ruwordnet.get_id_by_name('ПОСЕТИТЕЛЬ'), topn=10):
     synset_name = ruwordnet.get_name_by_id(synset_id)
     print(synset_name, similarity, ruwordnet.get_name_by_id(ruwordnet.get_hypernyms_by_id(synset_id)[0]))


Synset: ПОСЕТИТЕЛЬ
Query's hypernyms: ЧЕЛОВЕК
----------
Most similar words (node2vec):
ЧИТАТЕЛЬ БИБЛИОТЕКИ 0.9824718832969666 ПОСЕТИТЕЛЬ
ЗАВСЕГДАТАЙ 0.9758737087249756 ПОСЕТИТЕЛЬ
РЕСПОНДЕНТ 0.9270171523094177 ЧЕЛОВЕК
ТОЛСТЯК 0.9252837300300598 ЧЕЛОВЕК
СПОРТИВНЫЙ АРБИТР 0.9185411930084229 ЧЕЛОВЕК
СВИДЕТЕЛЬ (ЛИЦО, ПРИСУТСТВУЮЩЕЕ ДЛЯ УДОСТОВЕРЕНИЯ) 0.9158520102500916 ОЧЕВИДЕЦ
СДАТЧИК (ЛИЦО, ПРОИЗВ. СДАЧУ ПРОДУКЦИИ, ИМУЩ-ВА) 0.915824830532074 ЧЕЛОВЕК
ПОДРАЖАТЕЛЬ 0.9141154885292053 ЧЕЛОВЕК
ИНТЕЛЛИГЕНТ 0.9134445786476135 ЧЕЛОВЕК
МЕДВЕДЬ, СЛОН (НЕУКЛЮЖИЙ ЧЕЛОВЕК) 0.9120302200317383 НЕУКЛЮЖИЙ ЧЕЛОВЕК
----------
ОБСЛУЖИВАНИЕ ПОСЕТИТЕЛЕЙ 0.6754873991012573 ОБСЛУЖИТЬ (УДОВЛЕТВОРИТЬ НУЖДЫ)
ГОСТЬ (ПОСТОРОННЕЕ ЛИЦО) 0.6593257188796997 ПОСТОРОННЕЕ ЛИЦО
ГОСТЬ (ТОТ, КТО ПОСЕЩАЕТ, НАВЕЩАЕТ) 0.6593257188796997 ЧЕЛОВЕК ПО РОЛИ
НЕЗНАКОМЕЦ 0.651034414768219 ПОСТОРОННЕЕ ЛИЦО
ЧИТАТЕЛЬ БИБЛИОТЕКИ 0.6327954530715942 ПОСЕТИТЕЛЬ
ПОСТОЯЛЕЦ 0.6144022345542908 ЖИТЕЛЬ
КЛИЕНТ, ПОТРЕБИТЕЛЬ УСЛУГ 0.5967244505882263 ПОТ

In [18]:
hyper_ids = []
for synset_id, similarity in word2vec_model.most_similar(ruwordnet.get_id_by_name('ПОСЕТИТЕЛЬ'), topn=10):
     synset_name = ruwordnet.get_name_by_id(synset_id)
     for hyper_id in ruwordnet.get_hypernyms_by_id(synset_id):
          hyper_ids.append(hyper_id)
for hyper_id in hyper_ids[:10]:
    print(ruwordnet.get_name_by_id(hyper_id))

ОБСЛУЖИТЬ (УДОВЛЕТВОРИТЬ НУЖДЫ)
ПОСТОРОННЕЕ ЛИЦО
ЧЕЛОВЕК ПО РОЛИ
ПОСТОРОННЕЕ ЛИЦО
ЧЕЛОВЕК
ПОСЕТИТЕЛЬ
ЖИТЕЛЬ
ПОТРЕБИТЕЛЬ
ПУТЕШЕСТВЕННИК
ЧЕЛОВЕК ПО РОЛИ


In [25]:
hyper_ids = []
for synset_id, similarity in node2vec_model.most_similar(ruwordnet.get_id_by_name('ПОСЕТИТЕЛЬ'), topn=10):
     synset_name = ruwordnet.get_name_by_id(synset_id)
     for hyper_id in ruwordnet.get_hypernyms_by_id(synset_id):
          hyper_ids.append(hyper_id)
for hyper_id in hyper_ids[:10]:
    print(ruwordnet.get_name_by_id(hyper_id))

ПОСЕТИТЕЛЬ
ПОСЕТИТЕЛЬ
ЧЕЛОВЕК
ЧЕЛОВЕК
ЧЕЛОВЕК
ОЧЕВИДЕЦ
ЧЕЛОВЕК
ЧЕЛОВЕК
ЧЕЛОВЕК
НЕУКЛЮЖИЙ ЧЕЛОВЕК


In [ ]:
def normalized(a, axis=-1, order=2):
    """Utility function to normalize the rows of a numpy array."""
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis)) # (N,)
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

def make_training_matrices(source_dictionary, target_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []

    for id, name in ruwordnet.get_all_synsets('N'):
        if ' ' not in name:
            word_embedding = source_dictionary[id]
            node_embedding = target_dictionary[id]
            source_matrix.append(word_embedding)
            target_matrix.append(node_embedding )
    return np.array(source_matrix), np.array(target_matrix)

def learn_transformation(source_matrix, target_matrix, normalize_vectors=True):
    """
    Source and target matrices are numpy arrays, shape
    (dictionary_length, embedding_dimension). These contain paired
    word vectors from the bilingual dictionary.
    """
    if normalize_vectors:
        source_matrix = normalized(source_matrix)
        target_matrix = normalized(target_matrix)

    product = np.matmul(source_matrix.transpose(), target_matrix)
    U, s, Vt = np.linalg.svd(product)
    return np.matmul(U, Vt)


In [ ]:
src_matrix, trg_matrix = make_training_matrices(word2vec_model, node2vec_model)
transform = learn_transformation(src_matrix, trg_matrix)

(10173, 300) (10173, 300)


In [36]:
vector = word2vec_model[f'{ruwordnet.get_id_by_name("ПОСЕТИТЕЛЬ")}']
hyper_ids = []
for synset_id, similarity in node2vec_model.similar_by_vector(vector, topn=10):
     synset_name = ruwordnet.get_name_by_id(synset_id)
     for hyper_id in ruwordnet.get_hypernyms_by_id(synset_id):
          hyper_ids.append(hyper_id)
for hyper_id in hyper_ids[:10]:
    print(ruwordnet.get_name_by_id(hyper_id))

ДОХОД
ЕДА, ПИЩА
ЕДА, ПИЩА
РАСТИТЕЛЬНЫЕ КОРМА
СИЛОС
ПРЕДМЕТ, ВЕЩЬ
ПЕРСОНАЖ
РАСТИТЕЛЬНЫЕ КОРМА
ПРИПРАВА
ПИЩЕВКУСОВЫЕ ПРОДУКТЫ


In [37]:
hyper_ids = []
for synset_id, similarity in node2vec_model.similar_by_vector(vector @ transform, topn=10):
     synset_name = ruwordnet.get_name_by_id(synset_id)
     for hyper_id in ruwordnet.get_hypernyms_by_id(synset_id):
          hyper_ids.append(hyper_id)
for hyper_id in hyper_ids[:10]:
    print(ruwordnet.get_name_by_id(hyper_id))

ПЕРЕСЕЛЕНЕЦ
ПОМОЩНИК (ЧЕЛОВЕК)
СПАСАТЕЛЬ
ПОМОЩНИК (ЧЕЛОВЕК)
УЧЕНИК (НАЧИНАЮЩИЙ РАБОТНИК)
ПОМОЩНИК (ЧЕЛОВЕК)
РАБОТНИК
ПЕРЕСЕЛЕНЕЦ
ПОСОБНИК ПРЕСТУПЛЕНИЯ
СПАСИТЕЛЬ, СПАСИТЕЛЬНИЦА


In [39]:
for synset_id, similarity in node2vec_model.similar_by_vector(vector @ transform, topn=10):
     synset_name = ruwordnet.get_name_by_id(synset_id)
     print(synset_name, similarity, ruwordnet.get_name_by_id(ruwordnet.get_hypernyms_by_id(synset_id)[0]))

БЕЖЕНЕЦ 0.622832179069519 ПЕРЕСЕЛЕНЕЦ
ПОВОДЫРЬ СЛЕПОГО 0.6207084059715271 ПОМОЩНИК (ЧЕЛОВЕК)
ГОРНОСПАСАТЕЛЬ 0.6138981580734253 СПАСАТЕЛЬ
ПОДМАСТЕРЬЕ 0.6108173131942749 ПОМОЩНИК (ЧЕЛОВЕК)
АССИСТЕНТ (ПОМОЩНИК) 0.6095923185348511 ПОМОЩНИК (ЧЕЛОВЕК)
ИММИГРАНТ 0.6052301526069641 ПЕРЕСЕЛЕНЕЦ
НАВОДЧИК (ПОСОБНИК ПРЕСТУПЛЕНИЯ) 0.6049227714538574 ПОСОБНИК ПРЕСТУПЛЕНИЯ
СПАСАТЕЛЬ 0.6035510301589966 СПАСИТЕЛЬ, СПАСИТЕЛЬНИЦА
СВАХА 0.6016280055046082 ЧЕЛОВЕК ПО СФЕРЕ ДЕЯТЕЛЬНОСТИ
ЧУЖАК, ПРИШЛЫЙ ЧЕЛОВЕК 0.6005294919013977 ПОСТОРОННЕЕ ЛИЦО


In [258]:
def apply_transform_to_dict(emb_dict, transform):
    """
    emb_dict: dict[token] -> np.array(d,)
    transform: np.array(d, d)
    returns: new dict[token] -> np.array(d,)
    """
    out = {}
    for key, vec in emb_dict.items():
        out[key] = vec @ transform   # (d,) @ (d,d) -> (d,)
    return out